The functions toy_model_new and toy_model_pop_solve are the same.\
The function toy_model_pop_outer take $w_1, w_2, p_1, p_2, Pop_1, Pop_2$ in inputs and return Outer_Loss as output.\
The function toy_model_pop_solve no longer uses the function toy_model_pop_solve but minimze Outer_Loss with the contraints of Loss close to 0.

In [2]:
Pkg.add("Optim")
Pkg.add("Ipopt")
Pkg.add("JuMP")
Pkg.add("Test")
Pkg.add("Printf")
Pkg.add("Plots")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed IrrationalConstants ──── v0.2.2
   Installed DataAPI ──────────────── v1.14.0
   Installed DiffRules ────────────── v1.13.0
   Installed Adapt ────────────────── v3.6.1
   Installed PositiveFactorizations ─ v0.2.4
   Installed DiffResults ──────────── v1.1.0
   Installed SpecialFunctions ─────── v2.2.0
   Installed NaNMath ──────────────── v1.0.2
   Installed Setfield ─────────────── v1.1.1
   Installed StaticArraysCore ─────── v1.4.0
   Installed StaticArrays ─────────── v1.5.21
   Installed Optim ────────────────── v1.7.5
   Installed ConstructionBase ─────── v1.5.1
   Installed ArrayInterface ───────── v7.4.3
   Installed NLSolversBase ────────── v7.8.3
   Installed OrderedCollections ───── v1.6.0
   Installed FillArrays ───────────── v1.0.0
   Installed LineSearches ─────────── v7.2.0
   Installed ForwardDiff ──────────── v0.10.35
   Installed ChainRulesCore ───────── v1.15.7
 

In [3]:
using Pkg, Optim, Ipopt, JuMP, Test, Printf, Plots

# Functions

In [4]:
# Function toy_model computes the loss function for given values of w_1, w_2, p_1, p_2, Pop_1, Pop_2
# and some parameters values

function toy_model_new(w_1::Real, w_2::Real, p_1::Real, p_2::Real, Pop_1::Real, Pop_2::Real,
     α_1, α_2, β_1, β_2, δ_1, δ_2)
     
    #---------------------------
    # FUNCTIONS
    #---------------------------
    ## Labor demand in m = 1, 2 ---
    LD_1::Real = 1.0 + α_1 * w_1 + α_2 * w_1^2
    LD_2::Real = 1.0 + α_1 * w_2 + α_2 * w_2^2
    ## Commutes between 1 & 2 ---
    M_12::Real = max( Pop_1 * (w_2 - w_1)/w_1 , 0)
    M_21::Real = max( Pop_2 * (w_1 - w_2)/w_2 , 0)
    ## Labor supply ---
    LS_1::Real = Pop_1 - M_12 + M_21
    LS_2::Real = Pop_2 - M_21 + M_12
    ## Housing demand of firms 
    Hf_1::Real = 1.0 + β_1 * p_1 + β_2 * p_1^2
    Hf_2::Real = 1.0 + β_1 * p_2 + β_2 * p_2^2
    ## Housing demand of households  
    Hh_1::Real = Pop_1 * (0.3 * (w_1) / p_1)
    Hh_2::Real = Pop_2 * (0.3 * (w_2) / p_2)
    ## Housing supply 
    HT_1::Real = 0.2 + δ_1 * p_1 + δ_2 * p_1^2
    HT_2::Real = 0.2 + δ_1 * p_2 + δ_2 * p_2^2
    
    #---------------------------
    # Equilibrium conditions
    #---------------------------
    ## We will search for wages and housing prices solving this equilibrium 

    ## Labor supply --
    Loss_1::Real = (LS_1 - LD_1)^2
    Loss_2::Real = (LS_2 - LD_2)^2
    ## Housing market clearing --
    Loss_3::Real = (Hf_1 + Hh_1 - HT_1)^2
    Loss_4::Real = (Hf_2 + Hh_2 - HT_2)^2
    
    Loss::Real = Loss_1 + Loss_2 + Loss_3 + Loss_4
    return Loss
    
end

toy_model_new (generic function with 1 method)

In [5]:
# Function toy_model_pop_solve finds equilibrium values for w_1, w_2, p_1, p_2, Pop_1, Pop_2
# for given values of parameters α_1, α_2, β_1, β_2, δ_1, δ_2

function toy_model_pop_solve(o_a_1::Real, o_a_2::Real, o_β_1::Real, o_β_2::Real, o_δ_1::Real, o_δ_2::Real)
    @printf "Begin solver execution\n"

    @printf "Initializing model in solver\n"
    toy = Model(Ipopt.Optimizer)        # define empty model solved by Ipopt algorithm

    register(toy, :toy_model_new, 12, toy_model_new; autodiff = true)

    @variable(toy, w_1, start = 100.)
    @variable(toy, w_2, start = 100.)
    @variable(toy, p_1, start = 15.)
    @variable(toy, p_2, start = 10.)
    @variable(toy, Pop_1, start = 100.)
    @variable(toy, Pop_2, start = 200.)
    @NLconstraint(toy, w_1 >= 0)
    @NLconstraint(toy, w_2 >= 0)
    @NLconstraint(toy, p_1 >= 0)
    @NLconstraint(toy, p_2 >= 0)
    @NLconstraint(toy, Pop_1 >= 0)
    @NLconstraint(toy, Pop_2 >= 0)
    @NLparameter(toy, α_1 == o_a_1)
    @NLparameter(toy, α_2 == o_a_2)
    @NLparameter(toy, β_1 == o_β_1)
    @NLparameter(toy, β_2 == o_β_2)
    @NLparameter(toy, δ_1 == o_δ_1)
    @NLparameter(toy, δ_2 == o_δ_2)

    # next, we set an objective function
    @NLobjective(toy, Min, toy_model_new(w_1, w_2, p_1, p_2, Pop_1, Pop_2, α_1, α_2, β_1, β_2, δ_1, δ_2))

    @printf "Model initialized\n"

    @printf "Launching optimization\n"
    JuMP.optimize!(toy)
    @printf "Optimization complete\n"
    @printf "Results\n"

    println("RESULTS:")
    println(" $(w_1) = $(JuMP.value(w_1)) ")
    println(" $(w_2) = $(JuMP.value(w_2)) ")
    println(" $(p_1) = $(JuMP.value(p_1)) ")
    println(" $(p_2) = $(JuMP.value(p_2)) ")
    println(" $(Pop_1) = $(JuMP.value(Pop_1)) ")
    println(" $(Pop_2) = $(JuMP.value(Pop_2)) ")

    return (JuMP.value(w_1), JuMP.value(w_2), JuMP.value(p_1), JuMP.value(p_2), JuMP.value(Pop_1), JuMP.value(Pop_2))

end

toy_model_pop_solve (generic function with 1 method)

In [6]:
# Function toy_model_pop_solve finds equilibrium values for w_1, w_2, p_1, p_2, Pop_1, Pop_2
# for given values of parameters α_1, α_2, β_1, β_2, δ_1, δ_2

function toy_model_pop_solve(XXX)
    o_a_1::Real, o_a_2::Real, o_β_1::Real, o_β_2::Real, o_δ_1::Real, o_δ_2::Real = XXX
    @printf "Begin solver execution\n"

    @printf "Initializing model in solver\n"
    toy = Model(Ipopt.Optimizer)        # define empty model solved by Ipopt algorithm

    register(toy, :toy_model_new, 12, toy_model_new; autodiff = true)

    @variable(toy, w_1, start = 100.)
    @variable(toy, w_2, start = 100.)
    @variable(toy, p_1, start = 10.)
    @variable(toy, p_2, start = 10.)
    @variable(toy, Pop_1, start = 100.)
    @variable(toy, Pop_2, start = 200.)
    @NLconstraint(toy, w_1 >= 0)
    @NLconstraint(toy, w_2 >= 0)
    @NLconstraint(toy, p_1 >= 0)
    @NLconstraint(toy, p_2 >= 0)
    @NLconstraint(toy, Pop_1 >= 0)
    @NLconstraint(toy, Pop_2 >= 0)
    @NLparameter(toy, α_1 == o_a_1)
    @NLparameter(toy, α_2 == o_a_2)
    @NLparameter(toy, β_1 == o_β_1)
    @NLparameter(toy, β_2 == o_β_2)
    @NLparameter(toy, δ_1 == o_δ_1)
    @NLparameter(toy, δ_2 == o_δ_2)

    # next, we set an objective function
    @NLobjective(toy, Min, toy_model_new(w_1, w_2, p_1, p_2, Pop_1, Pop_2, α_1, α_2, β_1, β_2, δ_1, δ_2))

    @printf "Model initialized\n"

    @printf "Launching optimization\n"
    JuMP.optimize!(toy)
    @printf "Optimization complete\n"
    @printf "Results\n"

    println("RESULTS:")
    println(" $(w_1) = $(JuMP.value(w_1)) ")
    println(" $(w_2) = $(JuMP.value(w_2)) ")
    println(" $(p_1) = $(JuMP.value(p_1)) ")
    println(" $(p_2) = $(JuMP.value(p_2)) ")
    println(" $(Pop_1) = $(JuMP.value(Pop_1)) ")
    println(" $(Pop_2) = $(JuMP.value(Pop_2)) ")

    return ([JuMP.value(w_1), JuMP.value(w_2), JuMP.value(p_1), JuMP.value(p_2), JuMP.value(Pop_1), JuMP.value(Pop_2)])

end

toy_model_pop_solve (generic function with 2 methods)

In [7]:
# Function toy_model_pop_outer computes the outer_loss function 
# for given values  w_1, w_2, p_1, p_2, Pop_1, Pop_2 

function toy_model_pop_outer(w_1_opt, w_2_opt, p_1_opt, p_2_opt, Pop_1_opt, Pop_2_opt)

    Outer_Loss_1::Real=(w_1_opt-w_1_target)^2
    Outer_Loss_2::Real=(w_2_opt-w_2_target)^2
    Outer_Loss_3::Real=(p_1_opt-p_1_target)^2
    Outer_Loss_4::Real=(p_2_opt-p_2_target)^2
    Outer_Loss_5::Real=(Pop_1_opt-Pop_1_target)^2
    Outer_Loss_6::Real=(Pop_2_opt-Pop_2_target)^2
    
    Outer_Loss = Outer_Loss_1 + Outer_Loss_2 + Outer_Loss_3 + Outer_Loss_4 + Outer_Loss_5 + Outer_Loss_6
    return Outer_Loss
    
end

toy_model_pop_outer (generic function with 1 method)

In [18]:
# Function toy_model_outer_solve the model using moments
# = finds parameters values which minimize the gap between observed moments and model moments,
# that is equilibrium values of w_1, w_2, p_1, p_2, Pop_1, Pop_2

function toy_model_outer_solve()   

    Outer_toy = Model(Ipopt.Optimizer)            # define empty model solved by Ipopt algorithm

    register(Outer_toy, :toy_model_pop_outer, 6, toy_model_pop_outer; autodiff = true)
    println("Model initialized\n")

    # @variable(Outer_toy, α_1, start = 1.)    -2.0   0.0   
    # @variable(Outer_toy, α_2, start = 1.)    .02    0.0   
    # @variable(Outer_toy, β_1, start = 1.0)   -2.0   0.0   
    # @variable(Outer_toy, β_2, start = -1.02) -.02   0.0   
    # @variable(Outer_toy, δ_1, start = 0.2)   0.2    0.0   
    # @variable(Outer_toy, δ_2, start = -.02)  -.02   0.0   
    
    @variable(Outer_toy, α_1, start = -2.0)
    @variable(Outer_toy, α_2, start = .0)
    @variable(Outer_toy, β_1, start = -1.0)
    @variable(Outer_toy, β_2, start = .0)
    @variable(Outer_toy, δ_1, start = .0)
    @variable(Outer_toy, δ_2, start = .0)
    
    @variable(Outer_toy, w_1, start = w_1_target)
    @variable(Outer_toy, w_2, start = w_2_target)
    @variable(Outer_toy, p_1, start = p_1_target)
    @variable(Outer_toy, p_2, start = p_2_target)
    @variable(Outer_toy, Pop_1, start = Pop_1_target)
    @variable(Outer_toy, Pop_2, start = Pop_2_target)
    @NLconstraint(Outer_toy, w_1 >= 0)
    @NLconstraint(Outer_toy, w_2 >= 0)
    @NLconstraint(Outer_toy, p_1 >= 0)
    @NLconstraint(Outer_toy, p_2 >= 0)
    @NLconstraint(Outer_toy, Pop_1 >= 0)
    @NLconstraint(Outer_toy, Pop_2 >= 0)
    
    tol = 0.0001/4
    
    @NLconstraint(Outer_toy, (Pop_1 - (max( Pop_1 * (w_2 - w_1)/w_1 , 0)) + (max( Pop_2 * (w_1 - w_2)/w_2 , 0)) - (1.0 + α_1 * w_1 + α_2 * w_1^2))^2 <= tol) # Loss_1 <= tol
    @NLconstraint(Outer_toy, (Pop_2 - (max( Pop_2 * (w_1 - w_2)/w_2 , 0)) + (max( Pop_1 * (w_2 - w_1)/w_1 , 0)) - (1.0 + α_1 * w_2 + α_2 * w_2^2))^2 <= tol) # Loss_2 <= tol
    @NLconstraint(Outer_toy, (1.0 + β_1 * p_1 + β_2 * p_1^2 + Pop_1 * (0.3 * (w_1) / p_1) - (0.2 + δ_1 * p_1 + δ_2 * p_1^2))^2 <= tol) # Loss_3 <= tol
    @NLconstraint(Outer_toy, (1.0 + β_1 * p_2 + β_2 * p_2^2 + Pop_2 * (0.3 * (w_2) / p_2) - (0.2 + δ_1 * p_2 + δ_2 * p_2^2))^2 <= tol) # Loss_4 <= tol
   
    
    # next, we set an objective function
    @NLobjective(Outer_toy, Min, toy_model_pop_outer(w_1, w_2, p_1, p_2, Pop_1, Pop_2))
       
    #At this stage JuMP has a mathematical representation of our model internalized
    JuMP.optimize!(Outer_toy)

    println("RESULTS:")
    println(" $(w_1) = $(JuMP.value(w_1)) ")
    println(" $(w_2) = $(JuMP.value(w_2)) ")
    println(" $(p_1) = $(JuMP.value(p_1)) ")
    println(" $(p_2) = $(JuMP.value(p_2)) ")
    println(" $(Pop_1) = $(JuMP.value(Pop_1)) ")
    println(" $(Pop_2) = $(JuMP.value(Pop_2)) ")
    println()
    println(" $(α_1) = $(JuMP.value(α_1)) ")
    println(" $(α_2) = $(JuMP.value(α_2)) ")
    println(" $(β_1) = $(JuMP.value(β_1)) ")
    println(" $(β_2) = $(JuMP.value(β_2)) ")
    println(" $(δ_1) = $(JuMP.value(δ_1)) ")
    println(" $(δ_2) = $(JuMP.value(δ_2)) ")
    
    return(JuMP.value(α_1), JuMP.value(α_2), JuMP.value(β_1), JuMP.value(β_2), JuMP.value(δ_1), JuMP.value(δ_2))
end           

toy_model_outer_solve (generic function with 2 methods)

In [19]:
# Function toy_model_outer_solve the model using moments
# = finds parameters values which minimize the gap between observed moments and model moments,
# that is equilibrium values of w_1, w_2, p_1, p_2, Pop_1, Pop_2

function toy_model_outer_solve(α_1_start, α_2_start, β_1_start, β_2_start, δ_1_start, δ_2_start, )   

    Outer_toy = Model(Ipopt.Optimizer)            # define empty model solved by Ipopt algorithm

    register(Outer_toy, :toy_model_pop_outer, 6, toy_model_pop_outer; autodiff = true)
    println("Model initialized\n")

    @variable(Outer_toy, α_1, start = α_1_start)
    @variable(Outer_toy, α_2, start = α_2_start)
    @variable(Outer_toy, β_1, start = β_1_start)
    @variable(Outer_toy, β_2, start = β_2_start)
    @variable(Outer_toy, δ_1, start = δ_1_start)
    @variable(Outer_toy, δ_2, start = δ_2_start)
    
    @variable(Outer_toy, w_1, start = 100.)
    @variable(Outer_toy, w_2, start = 100.)
    @variable(Outer_toy, p_1, start = 10.)
    @variable(Outer_toy, p_2, start = 10.)
    @variable(Outer_toy, Pop_1, start = 100.)
    @variable(Outer_toy, Pop_2, start = 100.)
    @NLconstraint(Outer_toy, w_1 >= 0)
    @NLconstraint(Outer_toy, w_2 >= 0)
    @NLconstraint(Outer_toy, p_1 >= 0)
    @NLconstraint(Outer_toy, p_2 >= 0)
    @NLconstraint(Outer_toy, Pop_1 >= 0)
    @NLconstraint(Outer_toy, Pop_2 >= 0)
    
    tol = 0.1/4
    
    @NLconstraint(Outer_toy, (Pop_1 - (max( Pop_1 * (w_2 - w_1)/w_1 , 0)) + (max( Pop_2 * (w_1 - w_2)/w_2 , 0)) - (1.0 + α_1 * w_1 + α_2 * w_1^2))^2 <= tol) # Loss_1 <= tol
    @NLconstraint(Outer_toy, (Pop_2 - (max( Pop_2 * (w_1 - w_2)/w_2 , 0)) + (max( Pop_1 * (w_2 - w_1)/w_1 , 0)) - (1.0 + α_1 * w_2 + α_2 * w_2^2))^2 <= tol) # Loss_2 <= tol
    @NLconstraint(Outer_toy, (1.0 + β_1 * p_1 + β_2 * p_1^2 + Pop_1 * (0.3 * (w_1) / p_1) - (0.2 + δ_1 * p_1 + δ_2 * p_1^2))^2 <= tol) # Loss_3 <= tol
    @NLconstraint(Outer_toy, (1.0 + β_1 * p_2 + β_2 * p_2^2 + Pop_2 * (0.3 * (w_2) / p_2) - (0.2 + δ_1 * p_2 + δ_2 * p_2^2))^2 <= tol) # Loss_4 <= tol
   
    
    # next, we set an objective function
    @NLobjective(Outer_toy, Min, toy_model_pop_outer(w_1, w_2, p_1, p_2, Pop_1, Pop_2))
       
    #At this stage JuMP has a mathematical representation of our model internalized
    JuMP.optimize!(Outer_toy)

    println("RESULTS:")
    println(" $(w_1) = $(JuMP.value(w_1)) ")
    println(" $(w_2) = $(JuMP.value(w_2)) ")
    println(" $(p_1) = $(JuMP.value(p_1)) ")
    println(" $(p_2) = $(JuMP.value(p_2)) ")
    println(" $(Pop_1) = $(JuMP.value(Pop_1)) ")
    println(" $(Pop_2) = $(JuMP.value(Pop_2)) ")
    println()
    println(" $(α_1) = $(JuMP.value(α_1)) ")
    println(" $(α_2) = $(JuMP.value(α_2)) ")
    println(" $(β_1) = $(JuMP.value(β_1)) ")
    println(" $(β_2) = $(JuMP.value(β_2)) ")
    println(" $(δ_1) = $(JuMP.value(δ_1)) ")
    println(" $(δ_2) = $(JuMP.value(δ_2)) ")
    
    return(JuMP.value(α_1), JuMP.value(α_2), JuMP.value(β_1), JuMP.value(β_2), JuMP.value(δ_1), JuMP.value(δ_2))
end           

toy_model_outer_solve (generic function with 2 methods)

# Tests
## test 1

In [20]:
w_1_target=80
w_2_target=200
p_1_target=30
p_2_target=70
Pop_1_target=80
Pop_2_target=200

200

In [21]:
α_1, α_2, β_1, β_2, δ_1, δ_2 = toy_model_outer_solve()
@printf "\nLa fonction Loss pour ces valeurs est de : "
toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2)

Model initialized

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       32
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       12
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       10
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        4

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 5.17e+05 1.01e+00   0.0 0.00e+00    - 

4.486358041688001e-5

In [12]:
α_1, α_2, β_1, β_2, δ_1, δ_2

(-4.5716150532907145, 0.04253955301648193, 29.921437903150892, -0.4112097622894154, -30.581626942164327, 0.16919105130603965)

In [ ]:
β_1_grid = β_1-0.1:0.02:β_1+0.5
β_2_grid = β_2-0.002:0.0002:β_2+0.002
grid = [[i;j;k;l;m;n] for i in α_1, j in α_2, k in β_1_grid, l in β_2_grid, m in δ_1, n in δ_2]
val = map(toy_model_pop_solve, grid)
# w_1, w_2, p_1, p_2, Pop_1, Pop_2 = toy_model_pop_solve(α_1, α_2, β_1, β_2, δ_1, δ_2)
# toy_model_new(w_1, w_2, p_1, p_2, Pop_1, Pop_2, α_1, α_2, β_1, β_2, δ_1, δ_2)

In [ ]:
Α = [val[i,j][k] for i in 1:31, j in 1:20, k in 1:2]
minimum(Α), maximum(Α)

In [ ]:
Β = [val[i,j][k] for i in 1:31, j in 1:20, k in 3:4]
minimum(Β), maximum(Β)

In [ ]:
Δ = [val[i,j][k] for i in 1:31, j in 1:20, k in 5:6]
minimum(Δ), maximum(Δ)

In [ ]:
gr()
x1 = w_1_target-30:w_1_target+50
x2 = w_2_target-50:w_2_target+50
x3 = p_1_target-20:p_1_target+50
x4 = p_2_target-30:p_2_target+50
x5 = Pop_1_target-50:Pop_1_target+50
x6 = Pop_2_target-50:Pop_2_target+50
plot(x1, @.toy_model_new(x1, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "w_1")
plot!(x2, @.toy_model_new(w_1_target, x2, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "w_2")
plot!(x3, @.toy_model_new(w_1_target, w_2_target, x3, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "p_1")
plot!(x4, @.toy_model_new(w_1_target, w_2_target, p_1_target, x4, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "p_2")
plot!(x5, @.toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, x5, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "Pop_1")
plot!(x6, @.toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, x6, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "Pop_2")
plot!([w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target], [0, 0, 0, 0, 0, 0] ; label = "target", line = :scatter)

In [ ]:
toy_model_outer_solve(α_1, α_2, β_1, β_2, δ_1, δ_2)

In [ ]:
toy_model_pop_solve(ans[1], ans[2], ans[3], ans[4], ans[5], ans[6])

In [ ]:
toy_model_outer_solve(-1.9, 0.018, -0.37, -0.52, 1.6, -0.52)

In [ ]:
toy_model_pop_solve(ans[1], ans[2], ans[3], ans[4], ans[5], ans[6])

In [ ]:
toy_model_outer_solve(-1.9, 0.018, -0.35, -0.52, 1.6, -0.52)

In [ ]:
toy_model_pop_solve(ans[1], ans[2], ans[3], ans[4], ans[5], ans[6])

In [ ]:
toy_model_outer_solve(-1.92, 0.0175, -0.345, -0.524, 1.6, -0.516)

In [ ]:
toy_model_pop_solve(ans[1], ans[2], ans[3], ans[4], ans[5], ans[6])

In [ ]:
toy_model_outer_solve(-1.92, 0.0176, -0.338, -0.524, 1.59, -0.516)

In [ ]:
toy_model_pop_solve(ans[1], ans[2], ans[3], ans[4], ans[5], ans[6])

In [ ]:
toy_model_outer_solve(-1.9, 0.018, -0.35, -0.52, 1.6, -0.52)

In [ ]:
toy_model_pop_solve(ans[1], ans[2], ans[3], ans[4], ans[5], ans[6])

In [ ]:
α_1, α_2, β_1, β_2, δ_1, δ_2

In [ ]:
w_1_target=200
w_2_target=80
p_1_target=70
p_2_target=30
Pop_1_target=200
Pop_2_target=80

In [ ]:
α_1_bis, α_2_bis, β_1_bis, β_2_bis, δ_1_bis, δ_2_bis = toy_model_outer_solve()
toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1_bis, α_2_bis, β_1_bis, β_2_bis, δ_1_bis, δ_2_bis)

In [ ]:
α_1 - α_1_bis, α_2 - α_2_bis, β_1 - β_1_bis, β_2 - β_2_bis, δ_1 - δ_1_bis, δ_2 - δ_2_bis

In [ ]:
w_1, w_2, p_1, p_2, Pop_1, Pop_2 = toy_model_pop_solve(α_1, α_2, β_1, β_2, δ_1, δ_2)
toy_model_new(w_1, w_2, p_1, p_2, Pop_1, Pop_2, α_1, α_2, β_1, β_2, δ_1, δ_2)

## test 2

In [22]:
w_1_target=150
w_2_target=100
p_1_target=100
p_2_target=70
Pop_1_target=80
Pop_2_target=200

200

In [23]:
α_1, α_2, β_1, β_2, δ_1, δ_2 = toy_model_outer_solve()
toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2)

Model initialized

This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       32
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       12
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:       10
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        4

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 2.29e+05 1.01e+00   0.0 0.00e+00    - 

52.09402930110029

In [24]:
toy_model_pop_solve(α_1, α_2, β_1, β_2, δ_1, δ_2)

Begin solver execution
Initializing model in solver
Model initialized
Launching optimization
This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_

(137.04453419704777, 95.41903235387737, 40.376626827004145, 96.86871537193674, 84.97067854168233, 166.47536639544361)

In [ ]:
α_1, α_2, β_1, β_2, δ_1, δ_2 = toy_model_outer_solve(0.4, 0.004, -2.14, 0.014, 1.13, -0.01)
toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2)

In [60]:
toy_model_pop_solve(1.05, -0.001, -0.1, 0.0, 0.77, -0.0)

Begin solver execution
Initializing model in solver
Model initialized
Launching optimization
This is Ipopt version 3.14.10, running with linear solver MUMPS 5.5.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        6
        inequality constraints with only lower bounds:        6
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_

(162.59286600651143, 116.48200854791779, 64.60312203893444, 85.8773854886932, 73.37995956240535, 181.64415696276393)

In [ ]:
gr()
x1 = w_1_target-50:w_1_target+50
x2 = w_2_target-30:w_2_target+50
x3 = p_1_target-50:p_1_target+50
x4 = p_2_target-20:p_2_target+50
x5 = Pop_1_target-50:Pop_1_target+50
x6 = Pop_2_target-50:Pop_2_target+50
plot(x1, @.toy_model_new(x1, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "w_1")
plot!(x2, @.toy_model_new(w_1_target, x2, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "w_2")
plot!(x3, @.toy_model_new(w_1_target, w_2_target, x3, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "p_1")
plot!(x4, @.toy_model_new(w_1_target, w_2_target, p_1_target, x4, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "p_2")
plot!(x5, @.toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, x5, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "Pop_1")
plot!(x6, @.toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, x6, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "Pop_2")
plot!([w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target], [0, 0, 0, 0, 0, 0] ; label = "target", line = :scatter)

In [ ]:
α_1, α_2, β_1, β_2, δ_1, δ_2 = toy_model_outer_solve()
toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2)

In [ ]:
w_1_target=100
w_2_target=150
p_1_target=70
p_2_target=100
Pop_1_target=200
Pop_2_target=80

In [ ]:
α_1, α_2, β_1, β_2, δ_1, δ_2 = toy_model_outer_solve()
toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2)

In [ ]:
w_1, w_2, p_1, p_2, Pop_1, Pop_2 = toy_model_pop_solve(α_1, α_2, β_1, β_2, δ_1, δ_2)
toy_model_new(w_1, w_2, p_1, p_2, Pop_1, Pop_2, α_1, α_2, β_1, β_2, δ_1, δ_2)

In [ ]:
w_1, w_2, p_1, p_2, Pop_1, Pop_2 = toy_model_pop_solve(α_1, α_2, β_1, β_2, δ_1, δ_2)
toy_model_new(w_1, w_2, p_1, p_2, Pop_1, Pop_2, α_1, α_2, β_1, β_2, δ_1, δ_2)

## test 3

In [ ]:
w_1_target=200
w_2_target=70
p_1_target=130
p_2_target=30
Pop_1_target=150
Pop_2_target=100

In [ ]:
α_1, α_2, β_1, β_2, δ_1, δ_2 = toy_model_outer_solve()
toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2)

In [ ]:
w_1, w_2, p_1, p_2, Pop_1, Pop_2 = toy_model_pop_solve(α_1, α_2, β_1, β_2, δ_1, δ_2)
toy_model_new(w_1, w_2, p_1, p_2, Pop_1, Pop_2, α_1, α_2, β_1, β_2, δ_1, δ_2)

In [ ]:
gr()
x1 = w_1_target-50:w_1_target+50
x2 = w_2_target-20:w_2_target+50
x3 = p_1_target-50:p_1_target+50
x4 = p_2_target-20:p_2_target+50
x5 = Pop_1_target-50:Pop_1_target+50
x6 = Pop_2_target-50:Pop_2_target+50
plot(x1, @.toy_model_new(x1, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "w_1")
plot!(x2, @.toy_model_new(w_1_target, x2, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "w_2")
plot!(x3, @.toy_model_new(w_1_target, w_2_target, x3, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "p_1")
plot!(x4, @.toy_model_new(w_1_target, w_2_target, p_1_target, x4, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "p_2")
plot!(x5, @.toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, x5, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "Pop_1")
plot!(x6, @.toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, x6, α_1, α_2, β_1, β_2, δ_1, δ_2) ; label = "Pop_2")
plot!([w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target], [0, 0, 0, 0, 0, 0] ; label = "target", line = :scatter)

## test 4

In [ ]:
w_1_target = 155
w_2_target = 146
p_1_target = 49
p_2_target = 45
Pop_1_target = 121
Pop_2_target = 104

In [ ]:
α_1, α_2, β_1, β_2, δ_1, δ_2 = toy_model_outer_solve()
toy_model_new(w_1_target, w_2_target, p_1_target, p_2_target, Pop_1_target, Pop_2_target, α_1, α_2, β_1, β_2, δ_1, δ_2)

In [ ]:
w_1, w_2, p_1, p_2, Pop_1, Pop_2 = toy_model_pop_solve(α_1, α_2, β_1, β_2, δ_1, δ_2)
toy_model_new(w_1, w_2, p_1, p_2, Pop_1, Pop_2, α_1, α_2, β_1, β_2, δ_1, δ_2)